https://stackoverflow.com/questions/34975972/how-can-i-make-a-video-from-array-of-images-in-matplotlib

In [ ]:
# Complicated way to import finis if not installed

import os
import sys
finis_path = "../tp" #Folder containing finis folder
finis_abs_path = os.path.abspath(finis_path)
sys.path.append(finis_abs_path)

import finis
import numpy as np
import scipy.sparse as sp
import scipy
import pyamg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

In [ ]:
%matplotlib notebook

### Question 12: Setup

In [ ]:
mx = 1
my = 9
mesh = finis.triangulate(geom='project', max_area=0.005, mx=mx, my=my, n=40)
fe_u = finis.fe_space(mesh, order=2, order_int=2)
fe_p = finis.fe_space(mesh, order=1, order_int=2)
finis.plot_mesh(mesh, fig=plt.figure())

if True:
    markers = fe_u['markers']
    free_upper_boundary = True

    if free_upper_boundary:
        markers = np.logical_and(markers, np.logical_or(fe_u['dof'][:,1] < 10.0, np.logical_or(fe_u['dof'][:,0] == 0, fe_u['dof'][:,0] == 2)))
    plt.plot(fe_u['dof'][markers==1,0], fe_u['dof'][markers==1,1], 'r+')
    plt.axis("equal")


assert np.array_equal(fe_u['integ'], fe_p['integ']), "FE Spaces not compatible"
assert np.array_equal(fe_u['w'], fe_p['w']), "FE integration weights not compatible"

print("DOF: {}".format(fe_u['dof'].shape[0]*2 + fe_p['dof'].shape[0]))

In [ ]:
def fem_solve(mx, my, vx, vy):
    mesh = finis.triangulate(geom='project', max_area=0.05, mx=mx, my=my, n=40)
    fe_u = finis.fe_space(mesh, order=2, order_int=2)
    fe_p = finis.fe_space(mesh, order=1, order_int=2)
    
    dim_u = fe_u['dof'].shape[0]
    dim_p = fe_p['dof'].shape[0]
    
    assert np.array_equal(fe_u['integ'], fe_p['integ']), "FE Spaces not compatible"
    assert np.array_equal(fe_u['w'], fe_p['w']), "FE integration weights not compatible"
    
    # Free upper boundary
    markers = fe_u['markers']
    h_bnd = np.logical_or(fe_u['dof'][:,0] == 0, fe_u['dof'][:,0] == 2)
    not_top_bnd = fe_u['dof'][:,1] < 10.0
    markers = np.logical_and(markers, np.logical_or(h_bnd, not_top_bnd))
    
    min_dist = min([mx, 2-mx, my, 10-my])
    
    # Setting up system
    f1 = lambda x,y: np.zeros_like(x)
    f2 = lambda x,y: -9.81*np.ones_like(x)
    
    # Particular solution
    r2_dof = (fe_u['dof'][:,0]-mx)**2 + (fe_u['dof'][:,1]-my)**2
    markers_circle = np.logical_and(markers, r2_dof <= (0.25+1e-6)**2)
    w_h = np.zeros((dim_u, ))
    w_h[markers_circle] = 1
    UP1 = np.zeros((dim_u, ))
    UP2 = np.zeros((dim_u, ))
    UP1[markers_circle] = vx
    UP2[markers_circle] = vy
    UHP = np.concatenate((UP1, UP2, np.zeros((dim_p, ))))        
    
    # Construct non-dirichlet RHS
    W = sp.spdiags(fe_u['w'], [0], m=fe_u['w'].size, n=fe_u['w'].size)
    A11 = fe_u['DUX'].transpose().dot(W).dot(fe_u['DUX']) + fe_u['DUY'].transpose().dot(W).dot(fe_u['DUY'])
    A22 = A11
    A12 = fe_u['DUY'].transpose().dot(W).dot(fe_u['DUX'])
    A21 = A12.transpose()
    A13 = -fe_u['DUX'].transpose().dot(W).dot(fe_p['U'])
    A23 = -fe_u['DUY'].transpose().dot(W).dot(fe_p['U'])
    A = sp.bmat([[A11, A12, A13], [A21, A22, A23], [A13.transpose(), A23.transpose(), None]], format='csr')
    assert np.allclose((A - A.transpose()).data, 0), "A is not numerically symmetric!"
    
    # Construct non-dirichlet LHS
    F1int = f1(fe_u['integ'][:,0], fe_u['integ'][:,1])
    F2int = f2(fe_u['integ'][:,0], fe_u['integ'][:,1])

    _F1 = np.concatenate((
        fe_u['U'].transpose().dot(W).dot(F1int),
        fe_u['U'].transpose().dot(W).dot(F2int),
        np.zeros(dim_p, )
    ))
    F = _F1 - A.dot(UHP)
    
    # Apply dirichlet boundary
    row = np.where(markers==0)[0]
    col = np.arange(row.size)
    data = np.ones((row.size, ), dtype=np.float)
    P_u = sp.csr_matrix((data, (row, col)), shape=(markers.size, row.size))
    dim_ud = P_u.shape[1]
    P = sp.bmat([[P_u, None, None], [None, P_u, None], [None, None, sp.eye(dim_p)]], format='csr')
    Ad = P.transpose().dot(A).dot(P)
    assert np.allclose((Ad - Ad.transpose()).data, 0), "Ad is not numerically symmetric!"
    Fd = P.transpose().dot(F)
    
    # Solve
    x = sp.linalg.spsolve(Ad, Fd)
    # x[2*dim_ud:] -= np.mean(x[2*dim_ud:]) # remove mean pressure
    
    # Transform back (Dirichlet)
    u_h = P.dot(x) + UHP
    u1_h = u_h[0:dim_u]
    u2_h = u_h[dim_u:2*dim_u]
    p_h = u_h[2*dim_u:]
    assert p_h.size == dim_p
    
    # Calculate Viscos force
    Fb_x = sp.bmat([[A11], [A21], [A13.transpose()]], format='csr').dot(w_h).dot(u_h)
    Fb_y = sp.bmat([[A12], [A22], [A23.transpose()]], format='csr').dot(w_h).dot(u_h)

    return fe_u, fe_p, u1_h, u2_h, p_h, Fb_x, Fb_y

def fem_plot(fe_u, fe_p, u1_h, u2_h, p_h):
    shading= 'flat' # flat or gouraud
    max_u = max([
        np.amax(u1_h),
        np.amax(u2_h),
    ])
    min_u = min([
        np.amin(u1_h),
        np.amin(u2_h),
    ])

    fig = plt.figure(figsize=(9,5))
    ax = fig.add_subplot(1, 3, 1)
    plt.tripcolor(fe_u['dof'][:,0], fe_u['dof'][:,1], u1_h, shading=shading, vmin=min_u, vmax=max_u)
    ax.set_title("U_x")
    ax.set_xlabel("x")
    plt.colorbar()

    fig.add_subplot(1, 3, 2, sharex=ax, sharey=ax)
    plt.tripcolor(fe_u['dof'][:,0], fe_u['dof'][:,1], u2_h, shading=shading, vmin=min_u, vmax=max_u)
    plt.title("U_y")
    plt.xlabel("x")
    plt.colorbar()

    ax = fig.add_subplot(1, 3, 3, sharex=ax, sharey=ax)
    plt.tripcolor(fe_p['dof'][:,0], fe_p['dof'][:,1], p_h,  shading=shading)
    plt.title("P")
    plt.xlabel("x")
    plt.colorbar()

    plt.tight_layout()
    plt.show()
    
def fem_velocity(m, my=5, dt=0.1):
    t = 0
    mx = 1
    vx = 0
    vy = 0
    
    ts = list()
    mxs = list()
    mys = list()
    vxs = list()
    vys = list()
    
    ts.append(t)
    mxs.append(mx)
    mys.append(my)
    vxs.append(vx)
    vys.append(vy)
    
    i = 0
    
    while True:
        print("mx = {:5.4f}, vx = {:5.4f}, my = {:5.4f}, vy = {:5.4f}".format(mx, vx, my, vy))
        try:
            fe_u, fe_p, u1_h, u2_h, p_h, Fb_x, Fb_y = fem_solve(mx, my, vx=vx, vy=vy)
            ay = (- Fb_y - 9.81*m) / m
            vy = vy + dt*ay
            my = my + vy*dt + 0.5*ay*dt**2
            t = t + dt
            
            ts.append(t)
            mxs.append(mx)
            mys.append(my)
            vxs.append(vx)
            vys.append(vy)
            
            if (my >= 9.0) or (my <= 1.0):
                break
                
            if np.abs(ay*dt) < 1e-6:
                i += 1
                if i > 20:
                    break
            
        except ValueError:
            print("Out of bounds")
            break
    
    return np.array(ts), np.array(mxs), np.array(mys), np.array(vxs), np.array(vys)

In [ ]:
dt = 0.1
mx = 1
my = 9
vx = 0
vy = 0
m = 1 # 0.20 would be the mass if it were fluid
fe_u, fe_p, u1_h, u2_h, p_h, Fb_x, Fb_y = fem_solve(mx, my, vx=vx, vy=vy)
fem_plot(fe_u, fe_p, u1_h, u2_h, p_h)
print("F_y = {}".format(Fb_y))
print("F_x = {}".format(Fb_x))

print("Old: mx = {:5.4f}, vx = {:5.4f}, my = {:5.4f}, vy = {:5.4f}".format(mx, vx, my, vy))
ay = (- Fb_y - 9.81*m) / m
vy = vy + dt*ay
my = my + vy*dt + 0.5*ay*dt**2
print("New: mx = {:5.4f}, vx = {:5.4f}, my = {:5.4f}, vy = {:5.4f}".format(mx, vx, my, vy))

In [ ]:
fe_u, fe_p, u1_h, u2_h, p_h, Fb_x, Fb_y = fem_solve(mx=mx, my=my, vx=vx, vy=vy)
fem_plot(fe_u, fe_p, u1_h, u2_h, p_h)
print("F_y = {}".format(Fb_y))
print("F_x = {}".format(Fb_x))

print("Old: mx = {:5.4f}, vx = {:5.4f}, my = {:5.4f}, vy = {:5.4f}".format(mx, vx, my, vy))
ay = (- Fb_y - 9.81*m) / m
vy = vy + dt*ay
my = my + vy*dt + 0.5*ay*dt**2
print("New: mx = {:5.4f}, vx = {:5.4f}, my = {:5.4f}, vy = {:5.4f}".format(mx, vx, my, vy))

In [ ]:
ts, mxs, mys, vxs, vys = fem_velocity(m=1, my=5, dt=0.01)

In [ ]:
plt.figure()
plt.subplot(1,2,1)
plt.plot(ts, mys, '+-')
plt.subplot(1,2,2)
plt.plot(ts, vys, '+-')
plt.show()